In [3]:
names=['Cecilia','Lise','Marie']
letters = [len(n) for n in names]
print(max(letters))

7


In [4]:
longest_name = None
max_letters = 0
for i in range(len(names)):
    count = letters[i]
    if count >max_letters:
        longest_name = names[i]
        max_letters = count
print(longest_name)

Cecilia


In [7]:
#使用enumerate改进
names=['Cecilia','Lise','Marie']
longest_name = None
max_letters = 0
for i,name in enumerate(names):
    count = letters[i]
    if count>max_letters:
        longest_name = name
        max_letters = count
print(longest_name,max_letters)

Cecilia 7


In [8]:
#NO11.使用zip生成器封装迭代器，从迭代器获取下一个值封装成元组
names=['Cecilia','Lise','Marie']
letters = [len(n) for n in names]
longest_name = None
max_letters = 0
for name,count in zip(names,letters):
    if count > max_letters:
        longest_name = name
        max_letters  = count
print(longest_name,max_letters)

Cecilia 7


In [ ]:
#NO12.不要在for、while循环后面写else块，判断两个数互质有用

In [11]:
#合理利用try/excpet/else/finally结构中的每个代码块
#finally块：既要阻止异常向上传播给调用方，又要在异常发生时执行清理工作，使用try-finally。另外一种作用：可靠关闭文件句柄。NO43
handle = open('test.txt')
try:
    data = handle.read()
    print(data)
finally:
    handle.close()

test


In [ ]:
#try，except，else 如果try没有发生异常，执行else
#else块缩减try的代码量，把没有发生异常所要执行的代码用try-else隔开

In [13]:
#尽量用异常表示特殊情况，而不要使用None
a = (3,4)
_,b = a#跳过第一个元素
print(b)

4


In [16]:
#如何在闭包里使用外围作用域的变量？
#https://blog.csdn.net/qq_35716085/article/details/136027158
def sort_priority(values,group):
    def helper(x):
        if i in group:
            return (0,x)
    values.sort(key=helper)
numbers = [8,3,1,2,5,4,7,6]
group = {2,3,5,7}
sort_priority(numbers,group)
print(numbers)
print(type(group))

[1, 2, 3, 4, 5, 6, 7, 8]
<class 'set'>


In [18]:
#排序的技术：https://docs.python.org/zh-cn/3/howto/sorting.html#sorting-techniques
def sort_priority2(numbers,group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0,x)
        return (1,x)
    numbers.sort(key = helper)
    return found
numbers = [8,3,1,2,5,4,7,6]
group = {2,3,5,7}
found = sort_priority2(numbers,group)
print("Found:",found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


In [22]:
'''
上述的Found为False为什么呢？
我们在闭包里对Found进行变量赋值，相当于在变量helper里面定义了名为found的新的变量
这种问题称为作用域bug scoping bug 
获取包内的数据请使用nonlocal
'''
def sort_priority3(numbers,group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0,x)
        return (1,x)
    numbers.sort(key = helper)
    return found
numbers = [8,3,1,2,5,4,7,6]
group = {2,3,5,7}
found = sort_priority3(numbers,group)
print("Found:",found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


In [26]:
#最明智的做法是将nonlocal封装成辅助类
class Sorter(object):
    def __init__(self,group):
        self.group = group
        self.found = False
    def __call__(self,x):
        if x in self.group:
            self.found = True
            return (0,x)
        return (1,x)
numbers = [8,3,1,2,5,4,7,6]
group = {2,3,5,7}
sorter = Sorter(group)
numbers.sort(key = sorter)
assert sorter.found is True

In [29]:
#NO16:用生成器来改写直接返回列表的函数
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index,letter in enumerate(text):
        if letter == ' ':
            result.append(index+1)
    return result
address = 'Four score and seven years ago ...'
result = index_words(address)
print(result[:3])

[0, 5, 11]


In [34]:
#使用生成器修改上面的代码：使用yield
#'yield关键字主要用于从一个函数中返回值，但与普通的return不同，yield使得函数变成一个生成器（generator'
def index_words_iter(text):
    if text:
        yield 0
    for index,letter in enumerate(text):
        if letter ==' ':
            yield index+1
result = list(index_words_iter(address))
print(result)

[0, 5, 11, 15, 21, 27, 31]


In [37]:
#一个例子
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset+=1
            if letter == ' ':
                yield offset

from itertools import islice
with open('test.txt','r') as f:
    it = index_file(f)
    results = islice(it,0,3)
    print(list(results))

[0, 5, 7]


In [39]:
# NO17:在参数上面迭代要多加小心 不明白
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100*value /total
        result.append(percent)
    return result
visits = [15,35,80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [42]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100*value /total
        result.append(percent)
    return result
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)
it = read_visits('test.txt')
percentages = normalize(it)
print(percentages) #结果为空，迭代器只能产生一轮结果

[]


In [49]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)
def normalize_copy(numbers):
    number = list(numbers) #copy the iterator
    total = sum(number)
    result = []
    for value in number:
        percent = 100*value /total
        result.append(percent)
    return result
it = read_visits('test.txt')
percentages = normalize_copy(it)
print(percentages) #结果为空，迭代器只能产生一轮结果

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [ ]:
#上面的例子固然好，但是如果待复制的迭代器含有大量数据，从而导致程序在赋值迭代器的时候耗尽内存而崩溃怎么办
#一种方法是通过参数来接受另外一个函数，函数每次调用后都能返回新的迭代器
def normalize_func(get_iter):
    total = sum(get_iter())
    result = []
    for value in get_iter():
        percent = 100*value /total
        result.append(percent)
    return result

In [53]:
# NO18.用数量可变的位置参数减少视觉杂讯
# 参数加*表示可选参数，列表加*表示元素视为位置参数
# 变长参数传递时候总是将其转化为元组。
# NO19.用关键字参数表达可选行为
def remainder(number,divisor):
    return number % divisor
assert remainder(20,7) == 6

In [64]:
#No20.用None和文档字符串来描述具有动态默认值的参数
import datetime
import time
def log(message,when = datetime.datetime.now()):
    print('%s:%s'%(when,message))
log('Hi,there')
time.sleep(2)
log('After sleep...')

2024-09-09 20:38:59.269695:Hi,there
2024-09-09 20:38:59.269695:After sleep...


In [68]:
#执行上面的代码，我们发现日志的时间是一样的，datetime.now()只执行了一次！！！只是在函数定义的时候执行了一次
# 实现真正的默认值，习惯把默认值设置为None
import datetime
import time
def log(message,when = None):
    when = datetime.datetime.now() if when is None else when
    print('%s:%s'%(when,message))
log('Hi,there')
time.sleep(1)
log('After sleep...')

2024-09-09 20:51:55.550546:Hi,there
2024-09-09 20:51:56.550550:After sleep...


In [ ]:
# 如果参数的实际默认值是mutable的，记得用Nonne作为形式上的默认值。